In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
import src.data as data
import src.utility as utility

In [4]:
steps_behind_event=4
steps_after_event=3

In [6]:
sensors = data.sensors()
weather = data.weather()

print('Merging speeds and events...')
e = newtest_event
se = utility.merge_speed_events(newtest_speeds, e)
print('Done')

# create the time windows for each event
print('Creating time windows for events...')
joined_df = utility.time_windows_event(se, mode=mode, steps_behind=steps_behind_event, steps_after=steps_after_event)

# add other dataframes
# - events
events_info = e.drop(['KEY','KEY_2','DATETIME_UTC','step_duration','START_DATETIME_UTC','END_DATETIME_UTC'],axis=1) \
                .groupby(['index']).first()
joined_df = joined_df.merge(events_info, how='left', left_on='event_index', right_index=True)
# - weather
joined_df = joined_df.merge(weather, how='left')
# - sensors
joined_df = joined_df.merge(sensors, how='left')

print('Aggregating events in samples...')
joined_df = joined_df.round(4).groupby('sample_id').agg({
    'KEY':'first',
    'KM':'first',
    'event_index':lambda x: x.values[steps_behind_event],
    'DATETIME_UTC':list,
    'SPEED_AVG':list, #[list, lambda x: x[0:event_beginning_step].dropna().mean()],
    'SPEED_SD':list,
    'SPEED_MAX':list,
    'SPEED_MIN':list,
    'N_VEHICLES':list,
    'EMERGENCY_LANE':'first',
    'LANES':'first',
    'ROAD_TYPE':'first',
    'EVENT_DETAIL':lambda x: x.values[steps_behind_event],
    'EVENT_TYPE':lambda x: x.values[steps_behind_event],
    'WEATHER': list,
    'DISTANCE': list,
    'TEMPERATURE': list,
    'MIN_TEMPERATURE': list,
    'MAX_TEMPERATURE': list
})

# split the last m measures in different columns
def split_prediction_fields(row, event_beginning_step):
    return pd.Series((
            row.DATETIME_UTC[:event_beginning_step], row.DATETIME_UTC[event_beginning_step:], 
            row.SPEED_AVG[:event_beginning_step],    row.SPEED_AVG[event_beginning_step:],
            row.SPEED_SD[:event_beginning_step],
            row.SPEED_MAX[:event_beginning_step],
            row.SPEED_MIN[:event_beginning_step],
            row.N_VEHICLES[:event_beginning_step],
            row.WEATHER[:event_beginning_step],
            row.DISTANCE[:event_beginning_step],
            row.TEMPERATURE[:event_beginning_step],
            row.MIN_TEMPERATURE[:event_beginning_step],
            row.MAX_TEMPERATURE[:event_beginning_step],
    ))

print('Splitting time steps into separate columns...')

columns_to_split = ['DATETIME_UTC','DATETIME_UTC_y',
                    'SPEED_AVG','SPEED_AVG_Y',
                    'SPEED_SD', 'SPEED_MAX', 'SPEED_MIN', 'N_VEHICLES', 'WEATHER', 'DISTANCE',
                    'TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE']
joined_df[columns_to_split] = joined_df.apply(split_prediction_fields, axis=1, event_beginning_step=steps_behind_event)

for col_name in columns_to_split:
    if col_name.upper().endswith('_Y'):
        new_cols = ['{}_{}'.format(col_name, i) for i in range(0, steps_after_event+1)]
    else:
        new_cols = ['{}_{}'.format(col_name, i) for i in range(-steps_behind_event, 0)]

    joined_df[new_cols] = pd.DataFrame(joined_df[col_name].values.tolist(), index=joined_df.index)

# removed the residual columns of lists
joined_df = joined_df.drop(columns_to_split, axis=1)

# drop the rows for which all speeds are NaNs
print('Dataset shape:', joined_df.shape)
print('Dropping not available speeds...')
joined_df.dropna(how='all', subset=[f'SPEED_AVG_{i}' for i in range(-steps_behind_event, 0)], inplace=True)
print('Dataset shape reduced to:', joined_df.shape)

# cast to int some columns
joined_df = joined_df.astype({'EMERGENCY_LANE': 'int', 'LANES': 'int',
                                'ROAD_TYPE': 'int', 'EVENT_DETAIL': 'int'})

caching sensors

sensors loaded in: 0.0129 s

shape of the dataframe is: (2171, 5)
caching resources/dataset/preprocessed/base_structure_df_weather.csv.gz
Merging speeds and events...


KeyError: 'DATETIME_UTC'